In [103]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import collections
import os
import glob

Joined stations into 1 csv. 

Stations with same id, different name, but same (lat, lon) have been standardized to have the same name 

Stations with same id, different name, different (lat, lon) have also been standardized to use the same name and same (lat, lon), but has added variable to show that this station has been relocated in the past @TODO

In [104]:
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))
df_dict = {}
for f in csv_files:
    df_dict[f.split("\\")[-1]] = pd.read_csv(f)

In [105]:
df_dict.keys()

dict_keys(['all_stations.csv', 'Copy of Divvy_Stations_2014-Q1Q2.csv', 'Divvy_Stations_2013.csv', 'Divvy_Stations_2014-Q3Q4.csv', 'Divvy_Stations_2015.csv', 'Divvy_Stations_2016_Q1Q2.csv', 'Divvy_Stations_2016_Q3.csv', 'Divvy_Stations_2016_Q4.csv', 'Divvy_Stations_2017_Q1Q2.csv', 'Divvy_Stations_2017_Q3Q4.csv', 'new.csv'])

In [106]:
for key in df_dict.keys():
    col_name = df_dict[key].columns
    print(col_name)

Index(['Unnamed: 0', 'id', 'name', 'latitude', 'longitude', 'is_relocated'], dtype='object')
Index(['id', 'name', 'latitude', 'longitude', 'dpcapacity', 'online date'], dtype='object')
Index(['id', 'name', 'latitude', 'longitude', 'dpcapacity', 'landmark',
       'online date'],
      dtype='object')
Index(['id', 'name', 'latitude', 'longitude', 'dpcapacity', 'dateCreated'], dtype='object')
Index(['id', 'name', 'latitude', 'longitude', 'dpcapacity', 'landmark'], dtype='object')
Index(['id', 'name', 'latitude', 'longitude', 'dpcapacity', 'online_date'], dtype='object')
Index(['id', 'name', 'latitude', 'longitude', 'dpcapacity', 'online_date'], dtype='object')
Index(['id', 'name', 'latitude', 'longitude', 'dpcapacity', 'online_date'], dtype='object')
Index(['id', 'name', 'city', 'latitude', 'longitude', 'dpcapacity',
       'online_date'],
      dtype='object')
Index(['id', 'name', 'city', 'latitude', 'longitude', 'dpcapacity',
       'online_date', 'Unnamed: 7'],
      dtype='object')
I

In [107]:
chosen_station_var = ["id", "name", "latitude", "longitude"]
df_dict_final = {}
row = 0
for key in df_dict.keys():
    df_dict_final[key] = df_dict[key].loc[:, chosen_station_var]
    row += df_dict_final[key].shape[0]

for key in df_dict_final.keys():
    col_name = df_dict_final[key].columns
    print(col_name)

print(row)

Index(['id', 'name', 'latitude', 'longitude'], dtype='object')
Index(['id', 'name', 'latitude', 'longitude'], dtype='object')
Index(['id', 'name', 'latitude', 'longitude'], dtype='object')
Index(['id', 'name', 'latitude', 'longitude'], dtype='object')
Index(['id', 'name', 'latitude', 'longitude'], dtype='object')
Index(['id', 'name', 'latitude', 'longitude'], dtype='object')
Index(['id', 'name', 'latitude', 'longitude'], dtype='object')
Index(['id', 'name', 'latitude', 'longitude'], dtype='object')
Index(['id', 'name', 'latitude', 'longitude'], dtype='object')
Index(['id', 'name', 'latitude', 'longitude'], dtype='object')
Index(['id', 'name', 'latitude', 'longitude'], dtype='object')
5410


In [108]:
final_df = pd.DataFrame()
for key in df_dict_final.keys():
    final_df = pd.concat([final_df, df_dict_final[key]]) 

final_df.loc[:, ['latitude', 'longitude']] = final_df.loc[:, ['latitude', 'longitude']].round(6)
final_df.head(10)

,id,name,latitude,longitude
0,2,Buckingham Fountain,41.872667,-87.623971
1,3,Shedd Aquarium,41.867226,-87.615355
2,4,Burnham Harbor,41.856268,-87.613348
3,5,State St & Harrison St,41.873958,-87.627739
4,6,DuSable Harbor,41.885042,-87.612795
5,7,Field Blvd & South Water St,41.886349,-87.617517
6,9,Leavitt St & Archer Ave,41.828792,-87.680604
7,11,Jeffery Blvd & 71st St,41.766638,-87.576450
8,12,South Shore Dr & 71st St,41.766409,-87.565688
9,13,Wilton Ave & Diversey Pkwy,41.932500,-87.652681


In [109]:
len(final_df['id'].unique()), len(final_df['name'].unique())

(586, 661)

In [110]:
ids = []
for k in collections.Counter(final_df['id']).keys():
    curr = final_df[final_df['id'] == k]
    if len(curr['name'].unique()) != 1:
        ids.append(curr['id'].unique()[0])

In [111]:
for i in ids:
    curr = final_df[final_df['id'] == i]
    no_dup_coord = curr[['latitude', 'longitude']].drop_duplicates()
    if (len(no_dup_coord) == 1):
        lst = list(final_df['name'][final_df['id'] == i])
        mode_name = max(set(lst), key=lst.count)
        print(mode_name)
        final_df['name'][final_df['id'] == i] = mode_name

Loomis St & Taylor St
Orleans St & Elm St
Clinton St & Polk St (*)
Ogden Ave & Congress Pkwy
MLK Jr Dr & Oakwood Blvd
MLK Jr Dr & 47th St
St. Clair St & Erie St
Damen Ave & Cortland St
Sangamon St & Washington Blvd
MLK Jr Dr & 29th St
Shore Dr & 55th St
Wallace St & 35th St
Halsted St & 35th St
Cornell Ave & Hyde Park Blvd
MLK Jr Dr & 56th St (*)
Pulaski Rd & Eddy St


C:\Users\henry\AppData\Local\Temp\ipykernel_63004\2263109749.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['name'][final_df['id'] == i] = mode_name
C:\Users\henry\AppData\Local\Temp\ipykernel_63004\2263109749.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['name'][final_df['id'] == i] = mode_name
C:\Users\henry\AppData\Local\Temp\ipykernel_63004\2263109749.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [112]:
final_df['name'][final_df['id'] == 6] = "DuSable Harbor"
final_df['latitude'][final_df['id'] == 6] = 41.885042
final_df['longitude'][final_df['id'] == 6] = -87.612795

C:\Users\henry\AppData\Local\Temp\ipykernel_63004\1726447716.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['name'][final_df['id'] == 6] = "DuSable Harbor"
C:\Users\henry\AppData\Local\Temp\ipykernel_63004\1726447716.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['latitude'][final_df['id'] == 6] = 41.885042
C:\Users\henry\AppData\Local\Temp\ipykernel_63004\1726447716.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [113]:
num_curr = 0
ids_r = []
for k in collections.Counter(final_df['id']).keys():
    curr = final_df[final_df['id'] == k]
    if len(curr['name'].unique()) != 1:
        ids_r.append(curr['id'].unique()[0])
        print(curr)

len(ids_r)

     id                      name   latitude  longitude
0     2       Buckingham Fountain  41.872667 -87.623971
0     2  Michigan Ave & Balbo Ave  41.872293 -87.624091
47    2       Buckingham Fountain  41.872667 -87.623971
357   2  Michigan Ave & Balbo Ave  41.872638 -87.623979
357   2  Michigan Ave & Balbo Ave  41.872638 -87.623979
52    2       Buckingham Fountain  41.881060 -87.619486
0     2       Buckingham Fountain  41.876393 -87.620328
0     2       Buckingham Fountain  41.872667 -87.623971
     id                  name   latitude  longitude
11   15  Racine Ave & 19th St  41.856453 -87.656471
134  15  Racine Ave & 18th St  41.858181 -87.656487
3    15  Racine Ave & 18th St  41.858181 -87.656487
3    15  Racine Ave & 19th St  41.856453 -87.656471
11   15  Racine Ave & 18th St  41.858166 -87.656495
377  15  Racine Ave & 18th St  41.858166 -87.656495
422  15  Racine Ave & 18th St  41.858166 -87.656495
422  15  Racine Ave & 18th St  41.858166 -87.656495
404  15  Racine Ave & 18th S

52

In [114]:
ids_r.sort()
ids_r

[2,
 15,
 16,
 17,
 35,
 53,
 55,
 58,
 72,
 80,
 84,
 88,
 94,
 97,
 109,
 110,
 114,
 163,
 165,
 191,
 194,
 196,
 198,
 212,
 217,
 232,
 238,
 273,
 285,
 287,
 304,
 311,
 315,
 321,
 331,
 344,
 372,
 402,
 408,
 414,
 436,
 437,
 480,
 483,
 493,
 544,
 557,
 561,
 574,
 578,
 582,
 613]

In [115]:
len(final_df['id'].unique()), len(final_df['name'].unique())

(586, 644)

Treating all stations that have been relocated as the latest name and coord, but add variable on it has been relocated

In [116]:
for id in ids_r:
    curr = final_df[final_df['id'] == id]
    print(curr)
    latest_name = list(curr["name"])[-1]
    latest_lat = list(curr["latitude"])[-1]
    latest_lon = list(curr["longitude"])[-1]
    print(latest_name, latest_lat, latest_lon)

    final_df.loc[final_df['id'] == id, 'name'] = latest_name    
    final_df.loc[final_df['id'] == id, 'latitude'] = latest_lat   
    final_df.loc[final_df['id'] == id, 'longitude'] = latest_lon

    print(final_df[final_df['id'] == id])
    print('\n')


     id                      name   latitude  longitude
0     2       Buckingham Fountain  41.872667 -87.623971
0     2  Michigan Ave & Balbo Ave  41.872293 -87.624091
47    2       Buckingham Fountain  41.872667 -87.623971
357   2  Michigan Ave & Balbo Ave  41.872638 -87.623979
357   2  Michigan Ave & Balbo Ave  41.872638 -87.623979
52    2       Buckingham Fountain  41.881060 -87.619486
0     2       Buckingham Fountain  41.876393 -87.620328
0     2       Buckingham Fountain  41.872667 -87.623971
Buckingham Fountain 41.872667 -87.623971
     id                 name   latitude  longitude
0     2  Buckingham Fountain  41.872667 -87.623971
0     2  Buckingham Fountain  41.872667 -87.623971
47    2  Buckingham Fountain  41.872667 -87.623971
357   2  Buckingham Fountain  41.872667 -87.623971
357   2  Buckingham Fountain  41.872667 -87.623971
52    2  Buckingham Fountain  41.872667 -87.623971
0     2  Buckingham Fountain  41.872667 -87.623971
0     2  Buckingham Fountain  41.872667 -87.623

In [117]:
new_df = final_df.drop_duplicates(subset=["id", "name"]).sort_values(by = "id")

In [118]:
is_relocated = [0] * len(new_df)
new_df['is_relocated'] = is_relocated
for id in ids_r:
    new_df.loc[new_df['id'] == id, 'is_relocated'] = 1

In [119]:
len(new_df)

586

In [120]:
sum(new_df['is_relocated'])

52

In [121]:
new_df.to_csv('all_stations.csv', index=False)

In [101]:
# empty output confirms no more duplicate id's
for k in collections.Counter(new_df['id']).keys():
    curr = new_df[new_df['id'] == k]
    if len(curr['name'].unique()) != 1:
        print(curr)
